In [1]:
import time
import sys
sys.path.append('/home/mapdadmin/abraham/caltrans-data-exploration/')

In [2]:
from configparser import ConfigParser
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from process_traffic_data import apply_custom_transformations
import data_processing.process_utils as utils

In [3]:
from omnisci_connector.omni_connect import OmnisciConnect
#import ibis

In [10]:
config_path = '/home/mapdadmin/abraham/ini_files/config.ini'
print("read configuration file %s" %config_path)
config = ConfigParser()
config.read(config_path)
print("Configuration file read.")


read configuration file /home/mapdadmin/abraham/ini_files/config.ini
Configuration file read.


In [11]:
# useful columns:
traffic_data_columns = ['timestamp_',
                        'station',
                        'district',
                        'freeway',
                        'direction',
                        'lane_type',
                        'station_length',
                        'samples',
                        'pct_observed',
                        'total_flow',
                        'occupancy',
                        'speed'
                        ]

traffic_meta_columns = ['ID',
                        'County',
                        'State_PM',
                        'Abs_PM',
                        'Latitude',
                        'Longitude',
                        'Lanes',
                        'Name']

In [12]:

### metadata section ###
traffic_meta_path = config.get('Paths', 'meta_path')

# read in the traffic metadata to pandas:
df_traffic_metadata = pd.read_csv(traffic_meta_path, sep='\t', usecols=traffic_meta_columns).set_index('ID')
df_traffic_metadata = df_traffic_metadata.rename(str.lower, axis='columns')
print("traffic metadata file read.")





traffic metadata file read.


In [13]:
def get_year(paths, year):
    return [f for f in paths if year in f]

In [19]:
# initial parameters for reading in traffic data
threshold = 0.05
interest_col = 'speed'
grouper = 'station'
batch_limit = 5 #if running in parallel, set to number of parallel threads
file_ext = '.txt'

In [20]:
### Traffic Section ###

# get the paths of relevant files for the data
csv_files = config.get('Paths', 'data_path')
file_paths = utils.get_file_names(csv_files, extension=file_ext)

#file_paths = get_year(file_paths, '2019')

file_paths = sorted(file_paths)

print("Number of traffic files: ",len(file_paths))


Number of traffic files:  48


In [21]:
def apply_time_transformations(df):
    df['timestamp_'] = pd.to_datetime(df['timestamp_'], infer_datetime_format=True)

    # add a rounded timestamp for grouping later on: (e.g. 12:46 --> 13:00)
    df = utils.rounded_timestamp(df=df, name_current_ts='timestamp_', name_rounded_ts='timestamp_rounded', round_by='H')

    df = utils.add_day_of_week(df, 'timestamp_')

    # add column with hour of day for each data point
    df['hour_of_day'] = df['timestamp_'].dt.hour

    # add column with day of year for each data point
    df['day_of_year'] = df['timestamp_'].dt.dayofyear
    
    return df

In [22]:
def apply_custom_transformations(df, interest_col, threshold, grouper):

    # drop nas
    df = utils.grouped_drop_na(df, threshold, grouper=grouper, col=interest_col)

    # lower all column names
    df = utils.lower_col_names(df)

    df['state_pm'] = utils.state_pm_to_numeric(df['state_pm'])
    
    df = apply_time_transformations(df)

    # drop all rows with na in absolute postmarker field
    # df = df.dropna(subset=['abs_pm'])

    # downcast all ints to save memory
    # df = utils.downcast_int(df, ['station', 'freeway', 'samples', 'total_flow', 'lanes', 'county'])

    # downcast all floats to save memory
    # df = utils.downcast_type(df)

    df = df.drop(['station_length','name', 'pct_observed','abs_pm'],axis=1)
    return df

In [24]:
df_out = []
# extract and traffic data in batches:
no_data_cols = list(range(len(traffic_data_columns)))
for i in range(0, len(file_paths), batch_limit):
#for i in range(0, batch_limit, batch_limit):
    df_batch = []
    for f in file_paths[i:i + batch_limit]:
        print("Processing file: ", f)
        
        t0 = time.time()
        temp = pd.read_csv(f, header=None, names=traffic_data_columns, usecols=no_data_cols)
        t1 = time.time()
        print("time to read_csv: %f" %(t1-t0))
        df_batch.append(temp)
        t2 = time.time()
        print("time to append: %f" %(t2-t1))
        
    df_extracted_traffic = pd.concat(df_batch, ignore_index=True)
    t3 = time.time()
    print("time to concat: %f" %(t3-t2))
    
    df_extracted_traffic = df_extracted_traffic.drop('district', axis=1)
    
    t4 = time.time()
    df_extracted_traffic = df_extracted_traffic.join(df_traffic_metadata, on='station')
    t5 = time.time()
    print("time to join: %f" %(t5-t4))
    try:
        df_transformed_traffic = apply_custom_transformations(df=df_extracted_traffic,
                                                              interest_col=interest_col,
                                                              threshold=threshold,
                                                              grouper=grouper)
        t6 = time.time()
        print("time to transform: %f" %(t6-t5))
        
        df_out.append(df_transformed_traffic) 
    except ValueError as ex:
        print(ex)
        print("Skipping Batch starting at: ", i)

Processing file:  /home/mapdadmin/abraham/data_caltrans/hour/d04_text_station_hour_2015_01.txt
time to read_csv: 3.675838
time to append: 0.000548
Processing file:  /home/mapdadmin/abraham/data_caltrans/hour/d04_text_station_hour_2015_02.txt
time to read_csv: 3.136148
time to append: 0.000124
Processing file:  /home/mapdadmin/abraham/data_caltrans/hour/d04_text_station_hour_2015_03.txt
time to read_csv: 3.689518
time to append: 0.000138
Processing file:  /home/mapdadmin/abraham/data_caltrans/hour/d04_text_station_hour_2015_04.txt
time to read_csv: 3.611367
time to append: 0.000135
Processing file:  /home/mapdadmin/abraham/data_caltrans/hour/d04_text_station_hour_2015_05.txt
time to read_csv: 3.763702
time to append: 0.000266
time to concat: 1.901461
time to join: 4.466563
time to transform: 52.598141
Processing file:  /home/mapdadmin/abraham/data_caltrans/hour/d04_text_station_hour_2015_06.txt
time to read_csv: 3.672598
time to append: 0.000305
Processing file:  /home/mapdadmin/abraham

In [25]:
df_full = pd.concat(df_out, ignore_index=True)

In [35]:
df_full.tail()

,timestamp_,station,freeway,direction,lane_type,samples,total_flow,occupancy,speed,county,state_pm,latitude,longitude,lanes,timestamp_rounded,day_of_week,day_of_week_num,hour_of_day,day_of_year
83695143,2018-12-31 23:00:00,422116,101,N,ML,600,2747.0,0.0242,71.9,81.0,0.42,37.456467,-122.133857,5.0,2018-12-31 23:00:00,Monday,0,23,365
83695144,2018-12-31 23:00:00,422161,101,S,ML,599,2439.0,0.0286,70.9,81.0,0.40,37.456130,-122.133712,5.0,2018-12-31 23:00:00,Monday,0,23,365
83695145,2018-12-31 23:00:00,422357,880,N,ML,360,1124.0,0.0229,67.6,85.0,3.61,37.358660,-121.906756,3.0,2018-12-31 23:00:00,Monday,0,23,365
83695146,2018-12-31 23:00:00,422359,880,N,ML,479,1301.0,0.0206,69.3,85.0,5.27,37.380936,-121.904008,4.0,2018-12-31 23:00:00,Monday,0,23,365
83695147,2018-12-31 23:00:00,422749,780,E,ML,240,387.0,0.0162,65.9,95.0,7.27,38.091824,-122.231958,2.0,2018-12-31 23:00:00,Monday,0,23,365


In [34]:
import pickle

pickle.dump( df_full, open( "df_hour2.p", "wb" ),protocol=4 )

In [ ]:
load = True
import pickle
if load:
    df = pickle.load(open( "df_testing.p", "rb" ) )
    print(df.head())

In [ ]:
# freeway 101
len(df.loc[df['freeway']==101].station.unique())

#Select highway 101
df_101 = df.loc[df['freeway']==101]

#select north direction
df_101N = df_101.loc[df_101['direction']=='N']

#stations ordered by mile marker
stations = df_101N[['station','state_pm']]


In [ ]:
stations = df_101N[['station','state_pm']]

In [ ]:
df_101.isna().sum()

# Step 1: simple time series

In [ ]:
# step 1: simple 
station_1 = df_101N.loc[df_101N['station']==400001]

cols = ['timestamp_','total_flow','occupancy','speed']
station_1 = station_1[cols]

station_1 = station_1.set_index('timestamp_')
cols = station_1.columns

In [ ]:
station_1.plot(subplots=True, figsize=(12, 6))

In [ ]:
def tsplot(y, title, lags=None, figsize=(10, 6)):
    fig = plt.figure(figsize=figsize)
    layout = (2, 2)
    ts_ax = plt.subplot2grid(layout, (0, 0))
    hist_ax = plt.subplot2grid(layout, (0, 1))
    acf_ax = plt.subplot2grid(layout, (1, 0))
    pacf_ax = plt.subplot2grid(layout, (1, 1))

    y.plot(ax=ts_ax)
    ts_ax.set_title(title, fontsize=12, fontweight='bold')
    y.plot(ax=hist_ax, kind='hist', bins=25)
    hist_ax.set_title('Histogram')
    sm.tsa.graphics.plot_acf(y, lags=lags, ax=acf_ax)
    sm.tsa.graphics.plot_pacf(y, lags=lags, ax=pacf_ax)
    sns.despine()
    plt.tight_layout()
    plt.show()
    return ts_ax, acf_ax, pacf_ax

In [ ]:
tsplot(station_1['speed'],'speed')

In [ ]:
station_1 = station_1.dropna()


In [ ]:
model = sm.tsa.VARMAX(train, order=(2, 0), trend='c')
model_result = model.fit(maxiter=1000, disp=False)
model_result.summary()

In [ ]:
#creating the train and validation set
train = station_1[:int(0.8*(len(station_1)))]
valid = station_1[int(0.8*(len(station_1))):]

#fit the model
from statsmodels.tsa.vector_ar.var_model import VAR

model = VAR(endog=train)
model_fit = model.fit()

# make prediction on validation
prediction = model_fit.forecast(model_fit.y, steps=len(valid))

In [ ]:
valid.plot()

In [ ]:
#converting predictions to dataframe
from sklearn.metrics import mean_squared_error
pred = pd.DataFrame(index=range(0,len(prediction)),columns=[cols])
for j in range(0,len(cols)):
    for i in range(0, len(prediction)):
        pred.iloc[i][j] = prediction[i][j]

#check rmse
for i in cols:
    print('rmse value for', i, 'is : ', np.sqrt(mean_squared_error(pred[i], valid[i])))

In [ ]:
pred.shape

In [ ]:
valid.shape

In [ ]:
pred['speed'].plot()


In [ ]:
valid['speed'].plot()

In [ ]:
#make final predictions
model = VAR(endog=station_1)
model_fit = model.fit()
yhat = model_fit.forecast(model_fit.y, steps=5)
print(yhat)

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg




In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error


In [ ]:
# process data
values = station_1.values


# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaler1 = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

scaled1 = scaler1.fit_transform(values[:,2].reshape(-1, 1))


reframed = series_to_supervised(scaled,int(60/5))
reframed.drop(['var2(t)','var1(t)'], axis=1, inplace=True)

In [ ]:
reframed.head()

# LSTM Start

In [ ]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers import LSTM

In [ ]:
#
# split into train and test sets
values = reframed.values
train = values[:int(0.8*(len(station_1))), :]
test = values[int(0.8*(len(station_1))):, :]
# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

In [ ]:
# design network
model = Sequential()
model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')
# fit network
history = model.fit(train_X, train_y, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)


In [ ]:
# plot history
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))


In [ ]:
# invert scaling for forecast
inv_yhat = np.concatenate((yhat, test_X[:, 1:]), axis=1)
inv_yhat = scaler1.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]

In [ ]:
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = np.concatenate((test_y, test_X[:, 1:]), axis=1)
inv_y = scaler1.inverse_transform(inv_y)
inv_y = inv_y[:,0]

In [ ]:
import math
# calculate RMSE
rmse = math.sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

In [ ]:
inv_yhat

In [ ]:
plt.plot(inv_y)
plt.plot(inv_yhat)